In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# import the neccessary libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [2]:
# load the data and label in separate lists
alphabets_directory = '/content/drive/MyDrive/Dataset/Alphabets'
data = []
label = []
i = 0
for alphabet in os.listdir(alphabets_directory):
    alphabets_path = os.path.join(alphabets_directory, alphabet)
    for image in os.listdir(alphabets_path):
        image_path = os.path.join(alphabets_path, image)
        img = load_img(image_path, target_size=(64, 64))
        img = img_to_array(img)
        data.append(img)
        label.append(i)
    i+=1

# print("-----Completed-----")

In [3]:
# convert the lists to numpy arrays
data = np.array(data)
label = np.array(label)

In [4]:
# check the size of the data and labels
print(data.shape)
print(label.shape)

(26040, 64, 64, 3)
(26040,)


In [5]:
# converting the numeral labels to array like label
label = to_categorical(label)

In [6]:
# shuffle the data
n = np.arange(26040)
np.random.shuffle(n)
data_shuffled = data[n]
label_shuffled = label[n]

print("---Shuffling Completed Succesfully---")

---Shuffling Completed Succesfully---


In [7]:
data_shuffled = data_shuffled/255.0

In [8]:
# split the data into train and test data
# train-size: 80%
# val-size: 20%
x_train, x_test, y_train, y_test = train_test_split(data_shuffled, label_shuffled, test_size=0.1, stratify=label, random_state=12)

In [9]:
# make a sequential model

model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(26, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 31, 31, 64)       256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 31, 31, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0

In [11]:
# compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# set callbacks
callback = EarlyStopping(
    monitor='loss',
    patience=3
)

In [13]:
# fit the train data into the model
b = model.fit(
    x_train, y_train,
    epochs=40,
    callbacks=[callback],
)

Epoch 1/40
733/733 [==============================] - 24s 18ms/step - loss: 0.0648 - accuracy: 0.9819
Epoch 2/40
733/733 [==============================] - 13s 18ms/step - loss: 0.0080 - accuracy: 0.9977
Epoch 3/40
733/733 [==============================] - 13s 18ms/step - loss: 6.7946e-04 - accuracy: 0.9998
Epoch 4/40
733/733 [==============================] - 13s 18ms/step - loss: 0.0192 - accuracy: 0.9959
Epoch 5/40
733/733 [==============================] - 13s 18ms/step - loss: 0.0078 - accuracy: 0.9986
Epoch 6/40
733/733 [==============================] - 13s 18ms/step - loss: 0.0091 - accuracy: 0.9986


In [14]:
len(b.history['loss'])

6

In [15]:
y_pred_custom = model.predict(x_test)

In [16]:
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_custom, axis=1))
acc*100

99.88479262672811

In [ ]:
# save the model
model.save('alphabet_model.h5')